In [ ]:
using CUDA
using FiniteDifferences
using Printf
using Juliana

# Config

In [ ]:
data_dir = "/data/user/bellotti_r/data"
patient_ID = "test_00"

In [ ]:
fiona_standalone_bin_path = "/data/user/bellotti_r/semester_project_planning_metrics/src/pyftpp/bin"
fiona_jar_path = "$fiona_standalone_bin_path/ch.psi.ftpp.standalone.planner-1.0.9.jar";

In [ ]:
output_dir = "../output/test/gradient"
mkpath(output_dir);

In [ ]:
gantry_angle = 0.f0
couch_angle = 0.f0
nozzle_extraction = 20.f0

# Load data

In [ ]:
ct_path, patient_data = Juliana.load_patient_data(data_dir, patient_ID);

# Calculate Dij matrix

In [ ]:
optimisation_mask, optimisation_points_before, optimisation_point_indices = Juliana.get_optimisation_points_from_prescription(
    patient_data.ct.grid,
    patient_data.prescriptions,
    patient_data.structures,
    checkerboard_skip_n_inslice=10,
    checkerboard_skip_n_slices=1,
    margin_skip_n_slices=1,
);

optimisation_grid = Juliana.get_optimisation_grid(
    optimisation_points_before,
    patient_data.ct.grid,
)

sum(optimisation_mask)

In [ ]:
# Dummy value.
target_dose = 1.f0

In [ ]:
coldest_target_name = Juliana.coldest_target(patient_data.prescriptions)[1]

In [ ]:
Dij, optimisation_points = Juliana.FionaStandalone.calculate_Dij(
    output_dir,
    ct_path,
    target_dose,
    patient_data.structures[coldest_target_name],
    fiona_standalone_bin_path,
    fiona_jar_path,
    optimisation_grid,
    [gantry_angle],
    [couch_angle],
    [nozzle_extraction],
    debugging=false,
    optimization_points=optimisation_points_before,
    doseResolution=minimum(patient_data.ct.grid.spacing),
);

In [ ]:
size(Dij)

# Build optimisation configuration

In [ ]:
@time config = Juliana.get_optimisation_configuration(
    patient_data.ct,
    patient_data.prescriptions,
    patient_data.structures,
    Dij,
    optimisation_point_indices,
);

In [ ]:
subloss_weights = Dict{String, Float32}(
    "ideal_dose_loss" => 1.f0,
    "maximum_loss" => 1.f0,
    "minimum_loss" => 1.f0,
    "normalisation_variance" => 1.f0,
)

for constraint in config.prescriptions.constraints
    if constraint.priority == Juliana.soft
        continue
    end
    
    # Skip OARs for which there is no optimisation point.
    if sum(config.structures[constraint.structure_name])== 0
        continue
    end

    if constraint.kind == Juliana.constraint_mean
        subloss_weights["$(constraint.structure_name)_mean_loss"] = 1f0
    elseif Juliana.is_maximum_constraint(constraint)
        subloss_weights["$(constraint.structure_name)_max_loss"] = 1f0
    end
end

# Calculate the loss gradient

In [ ]:
w = ones(Float32, size(Dij, 2));
size(w)

In [ ]:
Juliana.loss(w, config, subloss_weights)

In [ ]:
@time Juliana.loss_gradient(w, config, subloss_weights);

In [ ]:
my_grad = Juliana.loss_gradient(w, config, subloss_weights);
my_grad_gpu = collect(Juliana.loss_gradient(cu(w), Juliana.to_gpu(config), subloss_weights));

@printf "Maximum difference between CPU and GPU gradient: %.2f%%" maximum(abs.(my_grad .- my_grad_gpu) ./ abs.(my_grad) * 100)

In [ ]:
my_grad

In [ ]:
grad_fd = FiniteDifferences.grad(FiniteDifferences.central_fdm(5, 1), w -> Juliana.loss(w, config, subloss_weights), w)[1];

maximum(abs.((my_grad .- grad_fd) ./ my_grad) * 100)

In [ ]:
grad_fd ./ my_grad

In [ ]:
my_grad

In [ ]:
grad_fd

In [ ]:
dose = config.Dij * w;
N = size(dose, 1);

In [ ]:
# # Calculate the dose normalisation gradient (if needed).
# J = Matrix{Float32}(undef, N, N)
# fill!(J, 0)

# normalisation_mean = mean_dose(dose, config.normalisationStructureMask)

# for i in 1:N
#     for j in 1:N
#         result = zero(T)
        
#         N_norm = sum(config.normalisationStructureMask)
#         if i == j
#             result += one(T) / normalisation_mean
#         end
#         result -= dose[i] * config.normalisationStructureMask[j] / (N_norm * normalisation_mean^2)
#         return config.normalisationDose * result
#         J[i, j] = Juliana.normalise_dose_jacobian(dose, config, i, j)
#     end
# end

In [ ]:
# N * N

In [ ]:
# sparse(J)

# Test the subloss parts

In [ ]:
ideal_loss_grad_juliana = Juliana.ideal_dose_loss_gradient(dose, config);
ideal_loss_grad_fd = FiniteDifferences.grad(FiniteDifferences.central_fdm(35, 1), dose -> Juliana.ideal_dose_loss(dose, config), dose)[1];

@assert maximum(abs.(ideal_loss_grad_juliana .- ideal_loss_grad_fd)) < 1e-3

In [ ]:
variance_loss_grad_juliana = Juliana.variance_dose_gradient(dose, config.normalisationStructureMask);
variance_loss_grad_fd = FiniteDifferences.grad(FiniteDifferences.central_fdm(35, 1), dose -> Juliana.variance_dose(dose, config.normalisationStructureMask), dose)[1];

@assert maximum(abs.(variance_loss_grad_juliana .- variance_loss_grad_fd)) < 1e-3

In [ ]:
minimum_grad_juliana = Juliana.minimum_loss_gradient(dose, config);
minimum_grad_fd = FiniteDifferences.grad(FiniteDifferences.central_fdm(35, 1), dose -> Juliana.minimum_loss(dose, config), dose)[1];

# The derivative is a step function, so we expect finite differences to fail.
@assert maximum(abs.(minimum_grad_juliana .- minimum_grad_fd)) < 1e-2

In [ ]:
maximum(dose)

In [ ]:
maximum(dose2)

In [ ]:
# TODO: Select a test case where the maximum loss is not zero...
dose2 = 1.1 * config.normalisationDose / maximum(dose) .* dose

maximum_grad_juliana = Juliana.maximum_loss_gradient(dose2, config);
maximum_grad_fd = FiniteDifferences.grad(FiniteDifferences.central_fdm(35, 1), dose -> Juliana.maximum_loss(dose, config), dose2)[1];

@assert Juliana.maximum_loss(dose2, config) > 0

maximum(abs.(maximum_grad_juliana .- maximum_grad_fd))

In [ ]:
Juliana.maximum_loss(dose2, config)

In [ ]:
sum(dose2 .> 1.05 * config.normalisationDose)

In [ ]:
sum(dose2[dose2 .> 1.05 * config.normalisationDose] .- 1.05 * config.normalisationDose) * 10

In [ ]:
sum(maximum_grad_fd .> 1e-1)

In [ ]:
maximum(maximum_grad_fd)

In [ ]:
maximum(maximum_grad_juliana)

In [ ]:
sum(maximum_grad_juliana .> 0)

In [ ]:
constr = patient_data.prescriptions.constraints[1]
mask = config.structures[constr.structure_name]
threshold = constr.dose
oar_mean_grad_juliana = Juliana.oar_mean_loss_gradient(dose, mask, threshold);
oar_mean_grad_fd = FiniteDifferences.grad(FiniteDifferences.central_fdm(35, 1), dose -> Juliana.oar_mean_loss(dose, mask, threshold), dose)[1];

@assert maximum(abs.(oar_mean_grad_juliana .- oar_mean_grad_fd)) == 0

In [ ]:
Juliana.oar_mean_loss(dose, mask)

In [ ]:
oar_mean_grad_juliana

# Simplified configuration to debug problems, if any

In [ ]:
n_voxels = 2

config_simple = Juliana.OptimisationConfiguration(
    1f0,                          # normalisation dose
    ones(Float32, n_voxels),      # normalisation structure
    ones(Float32, n_voxels),      # CT
    ones(Float32, n_voxels),      # ideal dose
    ones(Float32, n_voxels),      # importance
    [0.5f0, 1.2f0],      # minimum dose
    ones(Float32, n_voxels, 3),   # Dij
    ones(Float32, 3, n_voxels),   # Dij.T
    Dict{String, Vector{Float32}}(),
    Juliana.Prescriptions(Vector{Tuple{String, Float32}}(undef, 0), Vector{Juliana.Constraint{Float32}}(undef, 0)),
);

dose_simple = [0.3f0, 2.5f0];

In [ ]:
simple_ideal_loss_grad_juliana = Juliana.minimum_loss(dose_simple, config_simple)

In [ ]:
minimum_grad_fd = FiniteDifferences.grad(FiniteDifferences.central_fdm(12, 1), dose -> Juliana.minimum_loss(dose_simple, config_simple), dose_simple)[1]

In [ ]:
Juliana.minimum_loss_gradient(dose_simple, config_simple)

In [ ]:
subloss_weights = Dict{String, Float32}(
    "ideal_dose_loss" => 1.f0,
    "maximum_loss" => 1.f0,
    "minimum_loss" => 1.f0,
    "normalisation_variance" => 1.f0,
)

In [ ]:
Juliana.dose_loss_gradient(dose_simple, config_simple, subloss_weights)